##### "Os" allows to go through all the system and "Shutil" allows to do actions with files or directory.

In [1]:
import os
import shutil
from typing import Generator, List

##### This function gets all the chapters and save them in an array.

In [2]:
def get_chapters(root_directory: str) -> list[str]:
    """ Get all chapters from the root directory. """
    chapters = []
    for act_root, sub_dirs, files in os.walk(root_directory):
        if not sub_dirs:
            chapters.append(act_root)
    return chapters

##### This function gets groups of 100 chapters and returns some lists of 100 elements.

In [3]:
def get_groups(list_of_chapters: List[str], 
                length_group: int) -> Generator[List, None, None]:
    """ Split the list of chapters into groups of a specified length. """
    for i in range(0, len(list_of_chapters), length_group):
        yield list_of_chapters[i:i + length_group]

In [ ]:
def move_chapters(groups: list[str], dest_directory: str) -> None:
    """ Move chapters to a new directory. """
    os.makedirs(dest_directory, exist_ok=True)
    
    # Move each group of chapters to a new directory
    for i, group in enumerate(groups):
        group_name = f"group_{i+1}"
        dest_group = os.path.join(dest_directory, group_name)
        os.makedirs(dest_group, exist_ok=True)
        
        # Move each chapter in the group to the new directory
        for chapter in group:
            name_chapter = os.path.basename(chapter)
            new_directory = os.path.join(dest_group, name_chapter) 
            shutil.copytree(chapter, new_directory)
            print(f"Moved {chapter} to {new_directory}")
            
        print(f"\nGroup {i + 1} completed ({len(group)} chapters) en {dest_group}.\n")

In [5]:
# Replace with your actual root directory
root_directory = "datasets/LibriSpeech/dev-clean" 
# Replace with your actual root directory
dest_directory = "datasets/LibriSpeech/chapter-groups" 

chapters = get_chapters(root_directory)
groups = list(get_groups(chapters, 100))
move_chapters(groups, dest_directory) 

PermissionError: [Errno 13] Permission denied: 'datasets/LibriSpeech/dev-clean\\1272\\128104'